In [ ]:
import torch
import torchvision
from torchvision import transforms
from torch.utils import data

def data_iter(batch_size):
    trans=transforms.ToTensor()
    train=torchvision.datasets.FashionMNIST(root="../data",transform=trans,train=True,download=True)
    test=torchvision.datasets.FashionMNIST(root="../data",transform=trans,train=False,download=True)
    return data.DataLoader(train,batch_size,True),data.DataLoader(test,batch_size,True)

batch_size=10
train_iter,test_iter=data_iter(batch_size)
for x,y in train_iter:
    print(x.shape,y.shape)
    break;

torch.Size([256, 1, 28, 28]) torch.Size([256])


In [16]:
from torch import nn

net=nn.Sequential(nn.Flatten(),nn.Linear(784,256),nn.ReLU(),nn.Linear(256,10))

def initParams(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,std=0.01)
        nn.init.zeros_(m.bias)

net.apply(initParams)

loss=nn.CrossEntropyLoss(reduction="none")

updater=torch.optim.SGD(net.parameters(),lr=0.01)

num_epoch=20

class Accumulator():
    def __init__(self,n):
        self.data=[0.0]*n
    
    def add(self,*args):
        self.data=[ a+ float(b) for a,b in zip(self.data,args)]
    
    def __getitem__(self,index):
        return self.data[index]

def accuracy(y_hat,y):
    cp=y_hat.argmax(axis=1)
    cmp=cp.type(y.dtype)==y
    return cmp.type(y.dtype).sum()

def train(net,loss,updater,epoch):
    net.train()
    for i in range(epoch):
        metric=Accumulator(3)
        for x,y in train_iter:
            y_hat=net(x)
            l=loss(y_hat,y)
            updater.zero_grad()
            l.mean().backward()
            updater.step()
            metric.add(l.sum(),accuracy(y_hat,y),y.numel())
        print(f'epoch:{i} loss:{metric[0]/metric[2]} acc:{metric[1]/metric[2]}')


train(net,loss,updater,num_epoch)






epoch:0 loss:2.140053281148275 acc:0.39531666666666665
epoch:1 loss:1.5055047470092773 acc:0.5917
epoch:2 loss:1.1073522274017333 acc:0.6462833333333333
epoch:3 loss:0.9283137954711914 acc:0.6698833333333334
epoch:4 loss:0.8351912604014079 acc:0.6902166666666667
epoch:5 loss:0.7774900259653728 acc:0.7120666666666666
epoch:6 loss:0.7357364435831706 acc:0.7324666666666667
epoch:7 loss:0.7018677738825481 acc:0.75015
epoch:8 loss:0.6723872384389241 acc:0.7642333333333333
epoch:9 loss:0.6469344544728597 acc:0.7760166666666667
epoch:10 loss:0.6249946431477864 acc:0.78585
epoch:11 loss:0.6055843504587809 acc:0.79425
epoch:12 loss:0.5891794801076253 acc:0.8001
epoch:13 loss:0.5744346445083618 acc:0.80555
epoch:14 loss:0.5616239491144817 acc:0.81
epoch:15 loss:0.5501547108968099 acc:0.8140833333333334
epoch:16 loss:0.5401418207804362 acc:0.8171666666666667
epoch:17 loss:0.5313076963424682 acc:0.8203333333333334
epoch:18 loss:0.5232442196528116 acc:0.8226666666666667
epoch:19 loss:0.515915139007